In [1]:
import os, time
start = time.time()
very_start = time.time()

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [3]:
import dask
from dask.distributed import Client, wait, LocalCluster
import dask.dataframe as dd

In [4]:

client = Client(n_workers=1, 
                       threads_per_worker=96,
                       memory_limit='480GB',ip='10.1.0.xxx')
dask.config.set(shuffle='disk')
dask.config.set({'temporary_directory': '/path/to/dask_tmp'})

# Load Train

In [5]:
%%time
path = '/path/to/bin/tmp/'
train = dd.read_parquet(f'/path/to/dask_tmp/train_10_raw/*.parquet')#,dtypes=dtypes)
valid = dd.read_parquet(f'/path/to/dask_tmp/valid_10_raw/*.parquet')#,dtypes=dtypes)

CPU times: user 100 ms, sys: 23.2 ms, total: 123 ms
Wall time: 112 ms


In [6]:
cols_drop = ['tweet','tweet_nortsign']
train = train.drop(cols_drop,axis=1)
valid = valid.drop(cols_drop,axis=1)

In [7]:
dtype_names = ['count_words', 'count_char', 'count_ats']
for col in dtype_names:
    train[col] = train[col].astype('int32')
    valid[col] = valid[col].astype('int32')

In [8]:
label_names = ['reply_timestamp', 'retweet_timestamp', 'retweet_with_comment_timestamp', 'like_timestamp']
DONT_USE = ['tweet_id','tweet_timestamp','engaged_with_user_account_creation','enaging_user_account_creation','engage_time',
            'fold','enaging_user_id','engaged_with_user_id', 'dt_dow',
            'engaged_with_user_account_creation', 'enaging_user_account_creation', 'elapsed_time',
             'present_links','present_domains']
DONT_USE += label_names
features = [c for c in train.columns if c not in DONT_USE]

# Train Model Validate
We will train on random 10% of first 5 days and validation on last 2 days

In [9]:
%%time

SAMPLE_RATIO = 0.5
SEED = 1

if SAMPLE_RATIO < 1.0:
    print(len(train))
    train = train.sample(frac=SAMPLE_RATIO,random_state=42)
    train, = dask.persist(train)
    train.head()
    print(len(train))

train = train.compute()
# train = train.compute()
Y_train = train[label_names]
train = train.drop(label_names,axis=1)

features = [c for c in train.columns if c not in DONT_USE]
print('Using %i features:'%(len(features)),train.shape[1])
np.asarray(features)

21749425
10874713
Using 121 features: 121
CPU times: user 11.6 s, sys: 12.9 s, total: 24.6 s
Wall time: 2min 40s


array(['hashtags', 'present_media', 'tweet_type', 'language',
       'engaged_with_user_follower_count',
       'engaged_with_user_following_count',
       'engaged_with_user_is_verified', 'enaging_user_follower_count',
       'enaging_user_following_count', 'enaging_user_is_verified',
       'engagee_follows_engager', 'len_hashtags', 'len_domains',
       'len_links', 'dt_hour', 'dt_minute', 'dt_second', 'count_words',
       'count_char', 'tw_uhash', 'tw_hash', 'count_ats', 'hash0', 'hash1',
       'tw_freq_hash', 'tw_first_word', 'tw_second_word', 'tw_last_word',
       'tw_llast_word', 'tw_len', 'TE_present_media_reply_timestamp',
       'TE_tweet_type_reply_timestamp', 'TE_language_reply_timestamp',
       'TE_engaged_with_user_id_reply_timestamp',
       'TE_enaging_user_id_reply_timestamp',
       'TE_present_media_retweet_timestamp',
       'TE_tweet_type_retweet_timestamp', 'TE_language_retweet_timestamp',
       'TE_engaged_with_user_id_retweet_timestamp',
       'TE_enaging_

In [10]:
SAMPLE_RATIO = 0.052 # VAL SET NOW SIZE OF TEST SET
SEED = 1
if SAMPLE_RATIO < 1.0:
    print(len(valid))
    valid = valid.sample(frac=SAMPLE_RATIO,random_state=42)
    valid, = dask.persist(valid)
    valid.head()
    print(len(valid))
    
valid = valid.compute()
Y_valid = valid[label_names]
valid = valid.drop(label_names,axis=1)

8585006
446420


In [21]:
import xgboost as xgb
print('XGB Version',xgb.__version__)

xgb_parms = { 
    'max_depth':8, 
    'learning_rate':0.1, 
    'subsample':0.8,
    'colsample_bytree':0.3, 
    'eval_metric':'logloss',
    'objective':'binary:logistic',
    'nthread':94,
    'tree_method':'hist',
    #'predictor' : 'gpu_predictor'
}


XGB Version 1.0.2


In [22]:
if train.columns.duplicated().sum()>0:
    raise Exception(f'duplicated!: { train.columns[train.columns.duplicated()] }')
print('no dup :) ')
print(f'X_train.shape {train.shape}')
print(f'X_valid.shape {valid.shape}')

no dup :) 
X_train.shape (10874713, 121)
X_valid.shape (446420, 121)


In [23]:
%%time

for col in train.columns:
    if train[col].dtype=='bool':
        train[col] = train[col].astype('int8')
        valid[col] = valid[col].astype('int8')

CPU times: user 11.4 ms, sys: 3 ms, total: 14.4 ms
Wall time: 13.8 ms


In [24]:
train.head()

,hashtags,present_media,tweet_type,language,engaged_with_user_follower_count,engaged_with_user_following_count,engaged_with_user_is_verified,enaging_user_follower_count,enaging_user_following_count,enaging_user_is_verified,engagee_follows_engager,len_hashtags,len_domains,len_links,dt_hour,dt_minute,dt_second,count_words,count_char,tw_uhash,tw_hash,count_ats,hash0,hash1,tw_freq_hash,tw_first_word,tw_second_word,tw_last_word,tw_llast_word,tw_len,TE_present_media_reply_timestamp,TE_tweet_type_reply_timestamp,TE_language_reply_timestamp,TE_engaged_with_user_id_reply_timestamp,TE_enaging_user_id_reply_timestamp,TE_present_media_retweet_timestamp,TE_tweet_type_retweet_timestamp,TE_language_retweet_timestamp,TE_engaged_with_user_id_retweet_timestamp,TE_enaging_user_id_retweet_timestamp,TE_present_media_retweet_with_comment_timestamp,TE_tweet_type_retweet_with_comment_timestamp,TE_language_retweet_with_comment_timestamp,TE_engaged_with_user_id_retweet_with_comment_timestamp,TE_enaging_user_id_retweet_with_comment_timestamp,TE_present_media_like_timestamp,TE_tweet_type_like_timestamp,TE_language_like_timestamp,TE_engaged_with_user_id_like_timestamp,TE_enaging_user_id_like_timestamp,TE_present_domains_language_engagee_follows_engager_tweet_type_present_media_engaged_with_user_is_verified_reply_timestamp,TE_engaged_with_user_id_tweet_type_language_reply_timestamp,TE_tw_first_word_tweet_type_language_reply_timestamp,TE_tw_last_word_tweet_type_language_reply_timestamp,TE_hash0_tweet_type_language_reply_timestamp,TE_hash1_tweet_type_language_reply_timestamp,TE_tw_uhash_tweet_type_language_reply_timestamp,TE_tw_hash_reply_timestamp,TE_tw_freq_hash_reply_timestamp,TE_present_media_tweet_type_language_engaged_with_user_is_verified_enaging_user_is_verified_engagee_follows_engager_reply_timestamp,TE_present_domains_present_media_tweet_type_language_reply_timestamp,TE_present_links_present_media_tweet_type_language_reply_timestamp,TE_hashtags_present_media_tweet_type_language_reply_timestamp,TE_present_domains_language_engagee_follows_engager_tweet_type_present_media_engaged_with_user_is_verified_retweet_timestamp,TE_engaged_with_user_id_tweet_type_language_retweet_timestamp,TE_tw_first_word_tweet_type_language_retweet_timestamp,TE_tw_last_word_tweet_type_language_retweet_timestamp,TE_hash0_tweet_type_language_retweet_timestamp,TE_hash1_tweet_type_language_retweet_timestamp,TE_tw_uhash_tweet_type_language_retweet_timestamp,TE_tw_hash_retweet_timestamp,TE_tw_freq_hash_retweet_timestamp,TE_present_media_tweet_type_language_engaged_with_user_is_verified_enaging_user_is_verified_engagee_follows_engager_retweet_timestamp,TE_present_domains_present_media_tweet_type_language_retweet_timestamp,TE_present_links_present_media_tweet_type_language_retweet_timestamp,TE_hashtags_present_media_tweet_type_language_retweet_timestamp,TE_present_domains_language_engagee_follows_engager_tweet_type_present_media_engaged_with_user_is_verified_retweet_with_comment_timestamp,TE_engaged_with_user_id_tweet_type_language_retweet_with_comment_timestamp,TE_tw_first_word_tweet_type_language_retweet_with_comment_timestamp,TE_tw_last_word_tweet_type_language_retweet_with_comment_timestamp,TE_hash0_tweet_type_language_retweet_with_comment_timestamp,TE_hash1_tweet_type_language_retweet_with_comment_timestamp,TE_tw_uhash_tweet_type_language_retweet_with_comment_timestamp,TE_tw_hash_retweet_with_comment_timestamp,TE_tw_freq_hash_retweet_with_comment_timestamp,TE_present_media_tweet_type_language_engaged_with_user_is_verified_enaging_user_is_verified_engagee_follows_engager_retweet_with_comment_timestamp,TE_present_domains_present_media_tweet_type_language_retweet_with_comment_timestamp,TE_present_links_present_media_tweet_type_language_retweet_with_comment_timestamp,TE_hashtags_present_media_tweet_type_language_retweet_with_comment_timestamp,TE_present_domains_language_engagee_follows_engager_tweet_type_present_media_engaged_with_user_is_verified_like_timestamp,TE_engaged_with_user

In [25]:
#label_names = ['reply_timestamp', 'retweet_timestamp', 'retweet_with_comment_timestamp', 'like_timestamp']

In [26]:
%%time
# TRAIN AND VALIDATE

NROUND = 500
VERBOSE_EVAL = 50
#ESR = 50
LR = [0.1,0.03,0.07,0.01]

#Like
#xgb_parms['learning_rate'] = LR[TARGET_id]    
oof = np.zeros((len(valid),len(label_names)))
preds = []
for i in range(4):
    if i<=1:
        NROUND = 300
    else:
        NROUND = 500
    name = label_names[i]
    print('#'*25);print('###',name);print('#'*25)
    xgb_parms['learning_rate'] = LR[i]   
    start = time.time(); print('Creating DMatrix...')
        
    dtrain = xgb.DMatrix(data=train,label=Y_train.iloc[:, i])
    dvalid = xgb.DMatrix(data=valid,label=Y_valid.iloc[:, i])
    print('Took %.1f seconds'%(time.time()-start))
             
    start = time.time(); print('Training...')
    model = xgb.train(xgb_parms, 
                           dtrain=dtrain,
                           #evals=[(dtrain,'train'),(dvalid,'valid')],
                           num_boost_round=NROUND,
                           #early_stopping_rounds=ESR,
                           verbose_eval=VERBOSE_EVAL) 
    print('Took %.1f seconds'%(time.time()-start))
        
    start = time.time(); print('Predicting...')
    #Y_valid[f'pred_{name}'] = xgb.dask.predict(client,model,valid)
    oof[:, i] += model.predict(dvalid)
    #preds.append(xgb.dask.predict(client,model,valid))
    print('Took %.1f seconds'%(time.time()-start))
        
    del model, dtrain, dvalid

#########################
### reply_timestamp
#########################
Creating DMatrix...
Took 29.6 seconds
Training...
Took 335.4 seconds
Predicting...
Took 0.2 seconds
#########################
### retweet_timestamp
#########################
Creating DMatrix...
Took 29.1 seconds
Training...
Took 354.4 seconds
Predicting...
Took 0.2 seconds
#########################
### retweet_with_comment_timestamp
#########################
Creating DMatrix...
Took 28.9 seconds
Training...
Took 519.4 seconds
Predicting...
Took 0.3 seconds
#########################
### like_timestamp
#########################
Creating DMatrix...
Took 28.7 seconds
Training...
Took 499.1 seconds
Predicting...
Took 0.4 seconds
CPU times: user 1d 13h 47min 10s, sys: 2min 10s, total: 1d 13h 49min 20s
Wall time: 30min 25s


In [27]:
yvalid = Y_valid[label_names].values

# Compute Validation Metrics

In [28]:
from sklearn.metrics import precision_recall_curve, auc, log_loss
from sklearn.metrics import average_precision_score

def compute_prauc(pred, gt):
  prec, recall, thresh = precision_recall_curve(gt, pred)
  prauc = auc(recall, prec)
  return prauc

def compute_AP(pred, gt):
  return average_precision_score(gt, pred)

def calculate_ctr(gt):
  positive = len([x for x in gt if x == 1])
  ctr = positive/float(len(gt))
  return ctr

def compute_rce(pred, gt):
    cross_entropy = log_loss(gt, pred)
    data_ctr = calculate_ctr(gt)
    strawman_cross_entropy = log_loss(gt, [data_ctr for _ in range(len(gt))])
    return (1.0 - cross_entropy/strawman_cross_entropy)*100.0

# FAST METRIC FROM GIBA
def compute_rce_fast(pred, gt):
    cross_entropy = log_loss(gt, pred)
    yt = np.mean(gt)     
    strawman_cross_entropy = -(yt*np.log(yt) + (1 - yt)*np.log(1 - yt))
    return (1.0 - cross_entropy/strawman_cross_entropy)*100.0

In [29]:
%%time
txt = ''
for i in range(4):
    ap = compute_AP(oof[:,i], yvalid[:, i])
    rce   = compute_rce_fast(oof[:,i], yvalid[:, i])
    txt_ = f"{label_names[i]:20} AP:{ap:.5f} RCE:{rce:.5f}"
    print(txt_)
    txt += txt_ + '\n'

reply_timestamp      AP:0.21047 RCE:20.12967
retweet_timestamp    AP:0.41480 RCE:22.02740
retweet_with_comment_timestamp AP:0.04506 RCE:10.30338
like_timestamp       AP:0.71223 RCE:18.40327
CPU times: user 800 ms, sys: 2.99 ms, total: 803 ms
Wall time: 777 ms


In [30]:
print('This notebook took %.1f minutes'%((time.time()-very_start)/60.))

This notebook took 71.4 minutes
